In [1]:
import quantfreedom as qf
import numpy as np
import pandas as pd

from quantfreedom.backtester.indicators.talib_ind import from_talib
from quantfreedom._typing import (
    pdFrame, pdSeries
)
prices = pd.read_csv('30min.csv', index_col='time')
rsi_ind = from_talib(
    func_name='rsi',
    df_prices=prices,
    cart_product=False,
    combos=False,
    timeperiod=[15, 20],
)


In [2]:
tester = 'asd'
if tester.lower() in ('open', 'high', 'low', 'close'):
    print('yes')

In [3]:
np.zeros((2,5))

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [62]:
my_choices=[
         ('1','first choice'),
         ('2','second choice'),
         ('3','third choice')
]
my_choices

[('1', 'first choice'), ('2', 'second choice'), ('3', 'third choice')]

In [63]:
numbers = [(0,39), (1,584), (2,485)]
colors = ['green', 'purple']
pd.MultiIndex.from_product([numbers, colors],
                           names=['number', 'color'])

MultiIndex([( (0, 39),  'green'),
            ( (0, 39), 'purple'),
            ((1, 584),  'green'),
            ((1, 584), 'purple'),
            ((2, 485),  'green'),
            ((2, 485), 'purple')],
           names=['number', 'color'])

In [78]:
asdf1 = np.random.random_integers(0,30, size=10)
asdf2 = np.random.random_integers(0,30, size=10)
np.where(asdf1 < asdf2, True, False)

array([False, False,  True,  True,  True,  True,  True,  True, False,
       False])

In [83]:
len(asdf1)

10

In [67]:
numbers = [(0,39), (1,584), (2,485)]
pd.MultiIndex.from_tuples(numbers,
                           names=['number', 'color'])

MultiIndex([(0,  39),
            (1, 584),
            (2, 485)],
           names=['number', 'color'])

In [47]:
ema_300 = np.random.random_integers(1,60, size=(10,3))
user_args = np.array([10,20,30])
new_array = np.empty((ema_300.shape[0], ema_300.shape[1] * user_args.size), np.bool_)
start = 0
end = ema_300.shape[1]

for col in range(user_args.size):
    new_array[:, start : end] = np.where(ema_300 < user_args[col], True, False)
    start = end
    end += ema_300.shape[1]
new_array

array([[False, False,  True,  True, False,  True,  True,  True,  True],
       [False,  True, False, False,  True,  True, False,  True,  True],
       [False, False,  True, False, False,  True, False, False,  True],
       [False, False, False, False, False, False, False,  True, False],
       [ True, False,  True,  True, False,  True,  True,  True,  True],
       [False, False, False,  True, False, False,  True, False,  True],
       [False, False, False, False, False, False,  True, False, False],
       [False, False, False, False, False,  True, False, False,  True],
       [False, False, False, False, False, False,  True, False, False],
       [False,  True, False, False,  True, False, False,  True, False]])

In [18]:
np.where(asdf < 20, True, False)

array([[False,  True, False],
       [False,  True,  True],
       [False, False,  True],
       [False, False, False],
       [False,  True,  True],
       [ True, False, False],
       [False, False, False],
       [False, False,  True],
       [False,  True,  True],
       [ True,  True,  True]])

In [86]:
yes = () + ((0,)+(1,2,3,4),)
yes = yes + ((0,)+(5,6,7,8),)
yes

((0, 1, 2, 3, 4), (0, 5, 6, 7, 8))

In [4]:
temp_bigger_df_array = np.asarray(np.random.random_integers(0,1, size=(10,6)),dtype=np.bool_)
temp_smaller_df_array = np.asarray(np.random.random_integers(0,1, size=(10,6)),dtype=np.bool_)
combine_array = np.full((10,6), 8)

In [8]:
temp_bigger_df_array

array([[ True, False, False,  True,  True,  True],
       [False, False,  True,  True, False, False],
       [False,  True, False,  True,  True, False],
       [ True,  True,  True,  True, False,  True],
       [ True,  True, False,  True,  True,  True],
       [False,  True, False,  True,  True, False],
       [ True,  True, False, False,  True,  True],
       [False,  True, False, False,  True,  True],
       [False,  True, False,  True,  True, False],
       [ True, False, False,  True,  True,  True]])

In [5]:
combine_array[:, 0] = np.logical_and(
                temp_bigger_df_array[:,0] == True,
                temp_smaller_df_array[:, 0] == True)
combine_array

array([[1, 8, 8, 8, 8, 8],
       [0, 8, 8, 8, 8, 8],
       [0, 8, 8, 8, 8, 8],
       [1, 8, 8, 8, 8, 8],
       [0, 8, 8, 8, 8, 8],
       [0, 8, 8, 8, 8, 8],
       [0, 8, 8, 8, 8, 8],
       [0, 8, 8, 8, 8, 8],
       [0, 8, 8, 8, 8, 8],
       [0, 8, 8, 8, 8, 8]])

In [6]:
temp_bigger_df_array[:,0]

array([ True, False, False,  True,  True, False,  True, False, False,
        True])

In [7]:
temp_smaller_df_array[:, 0]

array([ True,  True,  True,  True, False, False, False,  True,  True,
       False])